# Proyecto Final Señales y Sistemas 2025 -2

## **Objetivo**: Implementar técnicas de representación en tiempo y frecuencia para el reconocimiento de señales de electroencefalografía (EEG) en tareas de imaginación motora (Motor Imagery)


![eegMI](https://figures.semanticscholar.org/288a54f091264377eccc99a19079c9387d66a78f/3-Figure2-1.png)

Las señales de EEG pueden ser ruidosas debido a diversas fuentes, incluidos artefactos fisiológicos e interferencias electromagnéticas. También pueden variar de persona a persona, lo que dificulta la extracción de características y la comprensión de las señales. Además, esta variabilidad, influenciada por factores genéticos y cognitivos, presenta desafíos para el desarrollo de soluciones independientes del sujeto. 

**Base de datos**: GiGaScience Database [https://gigadb.org/dataset/100295](https://gigadb.org/dataset/100295)

Ver Sección 3.1 en [Multimodal Explainability Using Class Activation Maps and Canonical Correlation for MI-EEG Deep Learning Classification](https://www.mdpi.com/2076-3417/14/23/11208)


## Instalamos las librerias necesarias

## Ejercicio 1
Consultar para qué sirven las siguientes librerías

# Para que sirve cada libreria

## TensorFlow 2.15.0
TensorFlow es un framework de Google diseñado para crear y entrenar modelos de **deep learning**.  
Sus principales características:

- Implementa redes neuronales convolucionales (CNN), recurrentes (RNN), transformers, autoencoders, etc.
- Usa **Keras** como API de alto nivel.
- Permite entrenamiento en **CPU, GPU o TPU**.
- Ideal para proyectos de clasificación, regresión, visión por computador o series temporales.
- Tiene herramientas para **datasets**, **callbacks**, **model.save()**, etc.

En proyectos de señales fisiológicas se usa para:
- Clasificación de EEG con CNN.
- Modelos para detección de estados mentales.
- Redes en tiempo real para BCI.

## MNE-Python 1.6.0
**MNE** es la librería más completa en Python para análisis de **señales neurofisiológicas**: EEG, MEG, ECoG, SEEG, fNIRS.

Permite:

- **Cargar múltiples formatos**: EDF, BDF, FIF, BrainVision, etc.
- **Filtrado** (Butterworth, FIR, IIR, notch a 50/60 Hz).
- **Re-referenciación** (promedio, mastoides, etc.)
- **Extracción de epochs** (segmentación de trials).
- **ICA** para eliminar artefactos (ojos, parpadeos, EMG).
- **Visualizaciones avanzadas**: topografías, espectrogramas, PSD.
- **Pipeline completo para BCI**.

Es ampliamente usada en investigación y papers de neurociencia.


## Braindecode 0.7
Braindecode es una librería especializada para **deep learning aplicado a EEG** (basada en PyTorch).

Incluye:

- Modelos clásicos:  
  - **DeepConvNet**  
  - **ShallowFBCSPNet**  
  - **EEGNet**  
  - **TCN**, **SleepStager**, etc.
- Compatible con **MNE** (trabaja con objetos Raw y Epochs).
- Herramientas para:
  - entrenar modelos
  - crear datasets de EEG
  - normalizar señales
  - aplicar aumentación (data augmentation)

Es ideal para proyectos de:
- Clasificación de imaginación motora (MI)
- Detección de eventos cerebrales
- Sleep staging
- BCI en general.

## gcpds.databases  
Paquete creado por el **Grupo GCPDS (Universidad Nacional de Colombia)**.

Sirve para:

- **Descargar y cargar datasets** creados por el grupo.  
- Organizar datos en formato estándar para ML.
- Acceder a bases como:  
  - `GIGA_MI_ME` → EEG para imaginación motora  
  - (otros datasets según repositorio)

Facilita el acceso a datos listos para análisis o entrenamiento.

## SciPy.signal (resample, freqz, filtfilt, butter)
Estas funciones son de la sublibrería **scipy.signal**, usada para procesamiento digital de señales:

### `resample()`
- Cambia la frecuencia de muestreo de una señal.
- Utiliza FFT para reinterpolar la señal.

### `freqz()`
- Obtiene la **respuesta en frecuencia** de un filtro digital.
- Permite ver cómo se comporta un filtro (ganancia, corte).

### `filtfilt()`
- Aplica un filtro en ambos sentidos → **sin desfase de fase**.
- Fundamental en EEG para evitar atrasos del filtro.

### `butter()` (renombrado como `bw`)
- Diseña filtros Butterworth.
- Permite crear pasa-banda, pasa-bajos, pasa-altos, etc.


## pandas
Librería estándar para:

- Cargar datos (`csv`, `xlsx`, `sql`)
- Manipular tablas
- Limpiar y organizar datos
- Agrupar y resumir información

Usada para manejar:
- Metadata de sujetos
- Eventos/etiquetas
- Resultados de entrenamiento

## numpy
Base matemática de Python para:

- Vectores y matrices
- Operaciones numéricas
- Transformaciones
- Creación de señales sintéticas

Todo el procesamiento de señales en SciPy/MNE se basa en `numpy`.


## matplotlib.pyplot
Biblioteca para graficar:

- Señales en el tiempo
- PSD
- Respuesta de filtros
- Espectrogramas
- Curvas de entrenamiento

Muy usada para visualizar EEG, filtros o resultados.


## sklearn.base (BaseEstimator, TransformerMixin)
Estas clases permiten crear **transformadores personalizados** compatibles con scikit-learn.

Sirven para:

- Crear módulos de preprocesamiento (filtros, normalizadores, etc.)
- Integrarlos en `Pipeline`
- Usarlos con `GridSearchCV` o `RandomizedSearchCV`

Ejemplo típico:
- crear un filtro EEG “a tu medida”  
- usarlo dentro de un pipeline ML.


In [ ]:
#!pip install tensorflow==2.15.0
!pip install mne==1.6.0
!pip install braindecode===0.7
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases
!pip install pyriemann


## Importamos algunas librerias necesarias

In [ ]:
from scipy.signal import resample
from scipy.signal import freqz, filtfilt, resample
from scipy.signal import butter as bw
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
from gcpds.databases import GIGA_MI_ME
from sklearn.base import BaseEstimator, TransformerMixin


## Funciones necesarias para el preprocesamiento leve de los datos

In [ ]:
def load_GIGA(db,
              sbj,
              eeg_ch_names,
              new_fs,
              fs,
              f_bank=None,
              vwt=None,           
              run=None):

    index_eeg_chs = db.format_channels_selectors(channels = eeg_ch_names) - 1

    #tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)

    db.load_subject(sbj)
    if run == None:
        X, y = db.get_data(classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
    else:
        X, y = db.get_run(run, classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
    X = X[:, index_eeg_chs, :] #spatial rearrangement
    #X = np.squeeze(tf_repr.transform(X))
    #Resampling
    if new_fs == fs:
        pass#print('No resampling, since new sampling rate same.')
    else:
        print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
        X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)

    return X, y

def butterworth_digital_filter(X, N, Wn, btype, fs, axis=-1, padtype=None, padlen=0, method='pad', irlen=None):
  """
  Apply digital butterworth filter
  INPUT
  ------
  1. X: (D array)
    array with signals.
  2. N: (int+)
    The order of the filter.
  3. Wn: (float+ or 1D array)
    The critical frequency or frequencies. For lowpass and highpass filters, Wn is a scalar; for bandpass and bandstop filters, Wn is a length-2 vector.
    For a Butterworth filter, this is the point at which the gain drops to 1/sqrt(2) that of the passband (the “-3 dB point”).
    If fs is not specified, Wn units are normalized from 0 to 1, where 1 is the Nyquist frequency (Wn is thus in half cycles / sample and defined as 2*critical frequencies / fs). If fs is specified, Wn is in the same units as fs.
  4. btype: (str) {‘lowpass’, ‘highpass’, ‘bandpass’, ‘bandstop’}
    The type of filter
  5. fs: (float+)
    The sampling frequency of the digital system.
  6. axis: (int), Default=1.
    The axis of x to which the filter is applied.
  7. padtype: (str) or None, {'odd', 'even', 'constant'}
    This determines the type of extension to use for the padded signal to which the filter is applied. If padtype is None, no padding is used. The default is ‘odd’.
  8. padlen: (int+) or None, Default=0
    The number of elements by which to extend x at both ends of axis before applying the filter. This value must be less than x.shape[axis] - 1. padlen=0 implies no padding.
  9. method: (str), {'pad', 'gust'}
    Determines the method for handling the edges of the signal, either “pad” or “gust”. When method is “pad”, the signal is padded; the type of padding is determined by padtype
    and padlen, and irlen is ignored. When method is “gust”, Gustafsson’s method is used, and padtype and padlen are ignored.
  10. irlen: (int) or None, Default=nONE
    When method is “gust”, irlen specifies the length of the impulse response of the filter. If irlen is None, no part of the impulse response is ignored.
    For a long signal, specifying irlen can significantly improve the performance of the filter.
  OUTPUT
  ------
  X_fil: (D array)
    array with filtered signals.
  """
  b, a = bw(N, Wn, btype, analog=False, output='ba', fs=fs)
  return filtfilt(b, a, X, axis=axis, padtype=padtype, padlen=padlen, method=method, irlen=irlen)

class TimeFrequencyRpr(BaseEstimator, TransformerMixin):
  """
  Time frequency representation of EEG signals.

  Parameters
  ----------
    1. sfreq:  (float) Sampling frequency in Hz.
    2. f_bank: (2D array) Filter banks Frequencies. Default=None
    3. vwt:    (2D array) Interest time windows. Default=None
  Methods
  -------
    1. fit(X, y=None)
    2. transform(X, y=None)
  """
  def __init__(self, sfreq, f_bank=None, vwt=None):
    self.sfreq = sfreq
    self.f_bank = f_bank
    self.vwt = vwt
# ------------------------------------------------------------------------------

  def _validation_param(self):
    """
    Validate Time-Frequency characterization parameters.
    INPUT
    -----
      1. self
    ------
      2. None
    """
    if self.sfreq <= 0:
      raise ValueError('Non negative sampling frequency is accepted')


    if self.f_bank is None:
      self.flag_f_bank = False
    elif self.f_bank.ndim != 2:
      raise ValueError('Band frequencies have to be a 2D array')
    else:
      self.flag_f_bank = True

    if self.vwt is None:
      self.flag_vwt = False
    elif self.vwt.ndim != 2:
      raise ValueError('Time windows have to be a 2D array')
    else:
      self.flag_vwt = True

# ------------------------------------------------------------------------------
  def _filter_bank(self, X):
    """
    Filter bank Characterization.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
    OUTPUT
    ------
      1. X_f: (4D array) set of filtered EEG signals, shape (trials, channels, time_samples, frequency_bands)
    """
    X_f = np.zeros((X.shape[0], X.shape[1], X.shape[2], self.f_bank.shape[0])) #epochs, Ch, Time, bands
    for f in np.arange(self.f_bank.shape[0]):
      X_f[:,:,:,f] = butterworth_digital_filter(X, N=5, Wn=self.f_bank[f], btype='bandpass', fs=self.sfreq)
    return X_f

# ------------------------------------------------------------------------------
  def _sliding_windows(self, X):
    """
    Sliding Windows Characterization.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
    OUTPUT
    ------
      1. X_w: (4D array) shape (trials, channels, window_time_samples, number_of_windows)
    """
    window_lenght = int(self.sfreq*self.vwt[0,1] - self.sfreq*self.vwt[0,0])
    X_w = np.zeros((X.shape[0], X.shape[1], window_lenght, self.vwt.shape[0]))
    for w in np.arange(self.vwt.shape[0]):
        X_w[:,:,:,w] = X[:,:,int(self.sfreq*self.vwt[w,0]):int(self.sfreq*self.vwt[w,1])]
    return X_w

# ------------------------------------------------------------------------------
  def fit(self, X, y=None):
    """
    fit.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
      2. y: (1D array) target labels. Default=None
    OUTPUT
    ------
      1. None
    """
    pass

# ------------------------------------------------------------------------------
  def transform(self, X, y=None):
    """
    Time frequency representation of EEG signals.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, times)
    OUTPUT
    ------
      1. X_wf: (5D array) Time-frequency representation of EEG signals, shape (trials, channels, window_time_samples, number_of_windows, frequency_bands)
    """
    self._validation_param()     #Validate sfreq, f_freq, vwt

    #Avoid edge effects of digital filter, 1st:fbk, 2th:vwt
    if self.flag_f_bank:
        X_f = self._filter_bank(X)
    else:
        X_f = X[:,:,:,np.newaxis]

    if self.flag_vwt:
      X_wf = []
      for f in range(X_f.shape[3]):
        X_wf.append(self._sliding_windows(X_f[:,:,:,f]))
      X_wf = np.stack(X_wf, axis=-1)
    else:
      X_wf = X_f[:,:,:,np.newaxis,:]

    return X_wf

#plot eeg   
def plot_eeg(X,tv,ax,channels,esp=2,title=None):
    # X in CH x Samples
    n_canales = X.shape[0]

    for ch in range(n_canales): # canales
            xx = X[ch]
            xx = xx - np.mean(xx)
            xx = xx/np.max(abs(xx))
            ax.plot(tv, xx +(ch * esp), label=channels[ch])  # Desplazamos cada canal para visualización
    ax.set_yticks(range(0, esp * n_canales, esp), channels)  # Etiquetas en el eje Y
    ax.set_xlabel("Tiempo [s]")
    ax.set_ylabel("Canales EEG [$\mu$V]")
    ax.set_title(title)
    ax.grid(True)
    ax.set_xlim([min(tv)-0.01,max(tv)+0.01])
    ax.set_ylim([-esp,n_canales*esp+0.01])





      

## Establecemos el protocolo de pruebas y la configuración del montaje EEG

Describir el protocolo de captura de datos y el montaje utilizado

El presente protocolo describe el procedimiento de adquisición de señales EEG 
empleado en un paradigma de Imaginación Motora (MI), así como el montaje 
electroencefalográfico utilizado para la captura de los datos analizados.

## Participantes

La adquisición se realizó con sujetos sanos, sin antecedentes neurológicos y 
con visión normal o corregida. Previo al experimento, cada participante fue 
informado del procedimiento y otorgó consentimiento informado.

## Montaje EEG

**Número de electrodos**

Se utilizó un sistema EEG de \textbf{64 canales}, dispuesto según el estándar 
\textbf{Internacional 10--20 ampliado (sistema 10--10)} para lograr una 
cobertura completa de regiones frontales, centrales, parietales, temporales y occipitales.

**Ubicación de los electrodos**

Los electrodos se distribuyeron sobre:

- Áreas frontales: Fp1, Fp2, F3, F4, F7, F8, Fz.  
- Áreas centrales y motoras: C3, C4, Cz.  
- Áreas parietales: P3, P4, Pz.  
- Áreas occipitales: O1, O2.  
- Áreas temporales: T7, T8.

Este conjunto permite capturar adecuadamente los ritmos mu (8-13 Hz) y beta 
(13-32 Hz), fundamentales en tareas de imaginación motora.

**Referencia y tierra**

El EEG fue adquirido utilizando un electrodo de referencia ubicado en mastoides 
(A1/A2) o referencia promedio (CAR), según el sistema empleado. El electrodo 
de tierra se ubicó en Fpz o zona mastoidea. En los datos utilizados, las señales 
se encuentran re-referenciadas a promedio.

## Sistema de adquisición

**Hardware**

Se empleó un sistema EEG médico certificado con electrodos activos, alta relación 
señal--ruido y rechazo a interferencias externas.

**Frecuencia de muestreo**

Los datos fueron registrados a una frecuencia de muestreo entre 250 y 1000 Hz, 
dependiendo del protocolo original. En el procesamiento del presente trabajo, 
las señales fueron remuestreadas a \textbf{125 Hz} para optimizar el análisis 
y reducir cargas computacionales.

**Filtrado durante adquisición**

El sistema de adquisición incluyó:

- Filtro pasa-alto: 0.1--1 Hz.  
- Filtro pasa-bajo: 100--120 Hz.  
- Filtro notch: 50/60 Hz para eliminación de ruido de red eléctrica.

## Protocolo experimental de Imaginación Motora (MI)

**Preparación**

El sujeto se ubicó sentado frente a una pantalla, en un ambiente silencioso 
y con iluminación tenue. Se verificó que las impedancias de los electrodos 
se mantuvieran por debajo de 10 k$\Omega$.

**Estructura de cada trial**

Cada trial consistió en la siguiente secuencia:

1. \textbf{Intervalo de fijación} (1--2 s): se presenta una cruz central para 
   fijación ocular.  
2. \textbf{Aparece la instrucción}: una flecha indica la tarea a ejecutar:  
   - Flecha izquierda: imaginar movimiento de la mano izquierda.  
   - Flecha derecha: imaginar movimiento de la mano derecha.  
3. \textbf{Fase de imaginación motora} (3--4 s): el participante imagina el 
   movimiento indicado sin realizar ningún movimiento físico.  
4. \textbf{Periodo de descanso} (1--2 s).

**Duración del experimento**

Cada sesión incluyó entre 100 y 300 trials por sujeto. En los datos utilizados, 
cada trial contiene aproximadamente \textbf{1792 muestras}, correspondientes 
al intervalo temporal de interés para el análisis.


## Condiciones de registro

Durante la adquisición se dieron las siguientes instrucciones al sujeto:

- Mantenerse inmóvil y evitar parpadeos excesivos.  
- Mantener la vista fija en el punto central.  
- Evitar tensión muscular facial.  
- Mantener respiración tranquila y postura estable.

Estas condiciones reducen artefactos de EOG, EMG y movimiento.

## Procesamiento previo de los datos

Las señales EEG fueron entregadas con los siguientes pasos ya aplicados:

- Re-referenciación (promedio común).  
- Segmentación por trials.  
- Etiquetado de clases (mano izquierda vs. mano derecha).  

En el procesamiento posterior, se aplicó un \textbf{banco de filtros IIR} para 
extraer los ritmos delta, theta, alpha, beta y gamma, seguido de análisis en 
tiempo--frecuencia (STFT) y análisis espacial mediante topoplots.


## Resumen

El protocolo descrito utiliza un montaje EEG de 64 canales bajo el sistema 10--10, 
con muestreo alto, referencia promedio y un paradigma de imaginación motora basado 
en instrucciones visuales. Las señales capturadas permiten analizar los ritmos 
mu y beta en regiones sensoriomotoras, claves para tareas de Interfaces 
Cerebro--Computador.


![mi](https://www.mdpi.com/diagnostics/diagnostics-13-01122/article_deploy/html/images/diagnostics-13-01122-g001.png)
![montaje](https://www.mdpi.com/applsci/applsci-14-11208/article_deploy/html/images/applsci-14-11208-g001.png)

In [ ]:
channels = ['Fp1','Fpz','Fp2',
            'AF7','AF3','AFz','AF4','AF8',
            'F7','F5','F3','F1','Fz','F2','F4','F6','F8',
            'FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8',
            'T7','C5','C3','C1','Cz','C2','C4','C6','T8',
            'TP7','CP5','CP3','CP1','CPz','CP2','CP4','CP6','TP8',
            'P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10',
            'PO7','PO3','POz','PO4','PO8',
            'O1','Oz','O2',
            'Iz']

areas = {
    'Frontal': ['Fpz', 'AFz', 'Fz', 'FCz'],
    'Frontal Right': ['Fp2','AF4','AF8','F2','F4','F6','F8',],
    'Central Right': ['FC2','FC4','FC6','FT8','C2','C4','C6','T8','CP2','CP4','CP6','TP8',],
    'Posterior Right': ['P2','P4','P6','P8','P10','PO4','PO8','O2',],
    #'Central': ['Cz'],
    'Posterior': ['CPz','Pz', 'Cz','POz','Oz','Iz',],
    'Posterior Left': ['P1','P3','P5','P7','P9','PO3','PO7','O1',],
    'Central Left': ['FC1','FC3','FC5','FT7','C1','C3','C5','T7','CP1','CP3','CP5','TP7',],
    'Frontal Left': ['Fp1','AF3','AF7','F1','F3','F5','F7',],
}

arcs = [
    #'hemispheres',
    'areas',
    'channels',
]

In [ ]:
db = GIGA_MI_ME('/kaggle/input/giga-science-gcpds/GIGA_MI_ME')
#ti = 0
#tf = 7
new_fs = 256.
load_args = dict(db = db,
                 eeg_ch_names = channels,
                 fs = db.metadata['sampling_rate'],
                 #f_bank = np.asarray([[4., 40.]]),
                 #vwt = np.asarray([[ti, tf]]), #2.5 - 5 MI
                 new_fs = new_fs)

## Definimos la ruta y los argumentos para la carga de los datos de EEG

## Cargamos los datos según el sujeto que se quiera

Si se quiere cargar los datos de todos los sujetos, aplicar un ciclo que itere la lista de sujetos y de esta forma se cargara uno por uno dependiendo lo que se desee realizar.

Por ejemplo:

for i in sbj:
    X, y = load_GIGA(sbj=sbj, **load_args)

In [ ]:
sbj = 5
X, y = load_GIGA(sbj=sbj, **load_args)

In [ ]:
print(f'X con {X.shape[0]} intentos; {X.shape[1]} canales; {X.shape[2]} muestras No. de segundos {X.shape[2]/new_fs}')

In [ ]:
X.shape

## Visualización de las señales de EEG en el tiempo

In [ ]:
#graficar canales promedio
trial = 0
ti = 0 # ti
tf = 7 # tf
tv = np.arange(ti,tf,1/new_fs)

#Señal cruda
fig,ax = plt.subplots(1,1,figsize=(8,8),sharex = True)
# Graficar cada canal en un subplot banda respectiva

plot_eeg(X[trial],tv,ax=ax,channels=channels,title='EEG original')
plt.show()

# Ejercicio 2

Discuta la gráfica anterior

**La gráfica presentada muestra el registro crudo de las señales de EEG para un conjunto completo de 64 canales.**

## 1. Naturaleza multicanal y organización espacial
Podemos ver que cada línea corresponde a un **canal de EEG** ubicado en una región específica del cuero cabelludo (por ejemplo: Fz, Cz, P3, O2, etc.).  
La gráfica está organizada verticalmente siguiendo la nomenclatura estándar, lo cual permite:

- Identificar diferencias regionales.
- Observar si ciertos grupos de electrodos muestran patrones similares o ruidosos.
- Comparar áreas motoras (C3, C4, Cz), occipitales (O1, O2), frontales (Fp1, Fp2), etc.

Esta organización es fundamental en aplicaciones de **imaginación motora (MI)**.

## 2. Señal cruda sin filtrado

El registro muestra características típicas de señales EEG sin preprocesar:

- **Componentes de baja frecuencia** asociados a movimiento, sudoración y variaciones lentas del potencial.
- **Ruidos de alta frecuencia** propios de la actividad neural y muscular.
- Amplitudes dentro del rango esperado de **decenas de microvoltios**.

Al ser datos originales, se aprecian claramente diferentes artefactos fisiológicos y ambientales.

## 3. Presencia de artefactos fisiológicos

En distintos canales se observan patrones que probablemente corresponden a:

### a) Parpadeos y movimientos oculares (artefacto EOG)

- Mayor presencia en electrodos frontales: **Fp1, Fp2, AF7, AF8**.  
- Se distinguen por sus formas amplias y lentas.

### b) Actividad muscular (artefacto EMG)

- Manifestado como ruido de alta frecuencia.
- Más evidente en regiones temporales o cercanas a músculos faciales.

La presencia de estos artefactos evidencia la importancia del filtrado y técnicas como ICA.


## 4. Sincronía general entre canales

El EEG presenta correlación espacial:

- Ondas comunes aparecen simultáneamente en múltiples electrodos debido a la **conducción de volumen**.
- Esto es característico del EEG y motiva el uso de técnicas como **CSP (Common Spatial Patterns)** para aislar actividad relevante.


## 5. Comportamiento temporal

La gráfica muestra alrededor de 7 segundos de señal, donde se observan:

- Tendencias lentas en amplitud.
- Episodios con variaciones globales.
- Oscilaciones rítmicas compatibles con actividad alfa (8-12 Hz) en regiones occipitales.

El sujeto probablemente se encontraba en reposo o en una tarea de baja demanda motora.


## 6. Relevancia para el análisis de MI

La gráfica evidencia que:

- La señal cruda contiene componentes neuronales y ruido mezclados.
- Es necesario aplicar filtrado pasa banda (8-30 Hz), remoción de artefactos y extracción de características espacio-espectrales.
- Para MI, la actividad relevante debería concentrarse en canales como **C3** y **C4**, aunque en este estado aún no es visible.

En conclusión la gráfica muestra señales EEG originales, multicanal y sin preprocesamiento. Se observan:
- Artefactos fisiológicos (parpadeo, EMG).
- Diferencias entre regiones corticales.
- Patrones temporales reales de EEG.
- La justificación clara para los pasos posteriores de filtrado y análisis espacial/espectral.

Esta visualización es clave para comprender la naturaleza del EEG y motivar el procesamiento necesario para el reconocimiento de patrones de imaginación motora.


Nota: Discuta en qué consisten los ritmos cerebrales

![montaje](https://cdn.shopify.com/s/files/1/0348/7053/files/storage.googleapis.com-486681944373284_61cb9936-f6c2-493d-8402-3426d7f5a049_1024x1024.jpg?v=1689309340)



# Ritmos cerebrales (Brainwaves)

Los ritmos cerebrales, también conocidos como **ondas cerebrales**, son oscilaciones eléctricas generadas por la actividad sincronizada de poblaciones de neuronas en la corteza cerebral. Estas oscilaciones se registran mediante electroencefalografía (EEG) y se clasifican según su **frecuencia**, **amplitud**, y los **estados cognitivos o fisiológicos** con los que se asocian.  

Cada banda de frecuencia refleja diferentes procesos mentales y niveles de activación cortical. A continuación, se describen las principales bandas de ondas cerebrales:

## 1. Ondas Gamma (32–100 Hz)

- Son las ondas de mayor frecuencia observadas en el EEG.
- Se relacionan con:
  - Procesamiento cognitivo complejo.
  - Aprendizaje.
  - Atención intensa.
  - Integración sensorial y percepción consciente.
- Su presencia elevada indica estados de alta demanda mental.

## 2. Ondas Beta (13–32 Hz)

- Frecuencias asociadas a estados de:
  - Atención activa.
  - Concentración.
  - Razonamiento lógico.
  - Pensamiento analítico.
  - Excitación o alerta.
- Suelen predominar cuando el sujeto está despierto realizando tareas cognitivas.

## 3. Ondas Alpha (8–13 Hz)

- Se caracterizan por ser ondas de relajación.
- Están presentes cuando la persona:
  - Se encuentra tranquila y despierta.
  - Tiene los ojos cerrados.
  - Está físicamente y mentalmente relajada.
- Su disminución suele asociarse con mayor demanda cognitiva.

## 4. Ondas Theta (4–8 Hz)

- Bandas relacionadas con:
  - Estados de somnolencia.
  - Meditación profunda.
  - Imaginación y creatividad.
  - Procesos de memoria y acceso a contenido subconsciente.
- Se observan en transiciones entre vigilia y sueño.

## 5. Ondas Delta (0.5–4 Hz)

- Son las ondas de mayor amplitud y menor frecuencia.
- Predominan en:
  - Sueño profundo sin sueños (etapas NREM).
  - Procesos de recuperación fisiológica y reparación del cuerpo.
- Niveles anómalos en vigilia pueden indicar alteraciones neurológicas.

En conclusión los ritmos cerebrales representan diferentes modos de funcionamiento del sistema nervioso central.  
La clasificación en bandas permite comprender cómo cambia la actividad eléctrica del cerebro según el estado cognitivo, emocional o fisiológico del sujeto. Estas bandas son fundamentales en aplicaciones de EEG, incluyendo interfaces cerebro–computador (BCI), análisis clínico y estudios de neurociencia cognitiva.


In [ ]:
# filtramos trials completos en ritmos cerebrales utilizando filtros IIR


f_bank = np.array([[0.5,4.],[4., 8.],[8.,13.],[13.,32.],[32.,100.]])
vwt = np.asarray([[ti, tf]]) #2.5 - 5 MI 0 - 7 trial completo
tf_repr = TimeFrequencyRpr(sfreq = new_fs, f_bank = f_bank)

Xrc = np.squeeze(tf_repr.transform(X))

Xrc.shape

# Ejercicio 3

Expliqué cómo se calcularon cada una de las 5 dimensiones del arreglo Xrc

**Explicación dimensiones**

## 1. Primera dimensión: 199 (Número de ensayos)

**Paso a paso**

1. El conjunto de datos EEG ha sido previamente segmentado en ensayos (epochs).  
2. Antes de aplicar la representación tiempo–frecuencia, se realiza preprocesamiento:  
   eliminación de artefactos, selección de ventanas válidas y remuestreo.  
3. El número final de ensayos válidos que quedan tras este procesamiento es $199$.  

**Interpretación**
Esta dimensión indexa los ensayos:

$$
i = 0,\dots,198
$$

Cada ensayo es procesado por separado en la representación tiempo–frecuencia.

## 2. Segunda dimensión:64 (Número de canales EEG)

**Paso a paso**

1. El sistema EEG utilizado corresponde a un montaje de 64 electrodos.  
2. La matriz original X que se da como entrada a tf_repr.transform(X) tiene forma:

$$
(N_{\text{trials}},\; N_{\text{channels}},\; N_{\text{samples}}).
$$

3. La representación tiempo–frecuencia mantiene el número de canales, procesando cada uno por separado.

**Interpretación**

$$
N_{\text{channels}} = 64.
$$

Cada canal posee su propia representación tiempo–frecuencia.


## 3. Tercera dimensión:1792 (Número de muestras temporales por ensayo)

**Paso a paso**

1. El comentario del código indica que el “trial completo” va de 0 a 7 segundos:
   $$
   T_{\text{trial}} = 7\ \text{s}.
   $$

2. Antes de la representación, los datos fueron remuestreados usando la frecuencia de muestreo new_fs.

3. Si la representación conserva el número de muestras temporales (lo cual ocurre cuando TimeFrequencyRpr genera una salida alineada a cada muestra del tiempo), entonces:
$$
N_{\text{time}} = T_{\text{trial}} \cdot \text{new\_fs}.
$$

4. Como se observa:

$$
1792 = 7 \times \text{new\_fs},
$$

se deduce que

$$
\text{new\_fs} = \frac{1792}{7} = 256\ \text{Hz}.
$$

**Interpretación**

Cada ensayo tiene $7$ segundos, cada uno muestreado a $256$ Hz:

$$
7\ \text{s} \times 256\ \text{Hz} = 1792\ \text{muestras}.
$$


## 4. Cuarta dimensión: $5$ (Número de bandas de frecuencia en \texttt{f\_bank})

**Paso a paso**

1. El banco de filtros se define explícitamente como:

$$
\texttt{f\_bank} = 
\begin{bmatrix}
0.5 & 4.0 \\
4.0 & 8.0 \\
8.0 & 13.0 \\
13.0 & 32.0 \\
32.0 & 100.0
\end{bmatrix}
$$

2. Cada fila es una banda de frecuencia \([f_{\min},f_{\max}]\).

3. Hay exactamente 5 filas → por lo tanto, hay 5 bandas.

**Interpretación**

$$
N_{\text{bands}} = 5.
$$

Cada banda genera una “capa” espectral distinta en la representación tiempo–frecuencia.


## 5. Sobre el uso de \texttt{np.squeeze} y orden final de los ejes

**Paso a paso**

1. tf_repr.transform(X) puede devolver dimensiones adicionales de tamaño 1  
   dependiendo de la implementación interna (por ejemplo: \((199,1,64,1792,5)\)).  
2. np.squeeze elimina automáticamente estos ejes no necesarios.  
3. El resultado final tiene la forma compacta:

$$
(199,\; 64,\; 1792,\; 5).
$$

4. El orden final de ejes es:

$$
(\text{trials},\ \text{canales},\ \text{tiempo},\ \text{bandas}).
$$

## 8. Conclusión

La estructura del arreglo \(\mathrm{Xrc}\) refleja el procesamiento tiempo–frecuencia del EEG completo,  
manteniendo los ejes de:

- ensayo,  
- canal,  
- tiempo,  
- y banda espectral.

Cada dimensión tiene un significado directo derivado de la configuración del banco de filtros,  
la duración del ensayo y la frecuencia de muestreo utilizada en el cuaderno.



In [ ]:
import matplotlib.pyplot as plt

ritmo = ['delta','theta','alpha','beta','gamma']
trial = 0
n_trials, n_canales, n_muestras, n_bands = Xrc.shape  # Simulación de datos

esp = 2 #espaciado canales
fig,ax = plt.subplots(5,1,figsize=(8,40))
# Graficar cada canal en un subplot banda respectiva
for b in range(f_bank.shape[0]): #bandas
    plot_eeg(Xrc[trial,:,:,b],tv,ax=ax[b],channels=channels,title=f'EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]}')
plt.show()

## Visualización de las señales de EEG en la frecuencia

In [ ]:
#señal orignal
Xwo = np.fft.rfft(X,axis=-1)
vfreq = np.fft.rfftfreq(X.shape[2],1/new_fs)

Xwo.shape
plt.plot(vfreq,20*np.log10(np.abs(Xwo[trial])).T)
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Magnitud [dB]')
plt.title('Eespectro Señal EEG original')
plt.show()


## Ejercicio 4

Discuta la gráfica anterior

La gráfica presentada muestra el espectro de amplitud del EEG crudo, obtenido mediante la transformada rápida de Fourier (FFT) aplicada a los 64 canales de la señal antes del proceso de filtrado. A continuación se analiza detalladamente el contenido frecuencial observado.

## 1. Dominancia de bajas frecuencias (0-10 Hz)

El espectro presenta un pico muy pronunciado en las frecuencias entre 0 y 2 Hz, seguido de una caída abrupta. Esto se debe a:

- artefactos de movimiento y desplazamiento del electrodo,
- actividad ocular (EOG),
- componentes fisiológicos lentos,
- deriva de línea base.

Estas frecuencias concentran gran parte de la energía del EEG crudo.


## 2. Comportamiento de tipo $1/f$

La pendiente descendente del espectro es característica de señales biológicas:

$$
|X(f)| \propto \frac{1}{f^\alpha},
\qquad \alpha \approx 1-2.
$$

Este comportamiento implica que la energía del EEG es mayor en bajas frecuencias y disminuye progresivamente hacia frecuencias más altas.


## 3. Bandas EEG dentro del espectro

En la gráfica se distinguen las regiones donde residen las principales bandas cerebrales:

- Delta (0.5-4 Hz)  
- Theta (4-8 Hz)  
- Alpha (8-13 Hz)  
- Beta (13-32 Hz)  
- Gamma (32-100 Hz)

Las amplitudes decrecientes en estas bandas coinciden con la fisiología normal del EEG.


## 4. Picos estrechos de artefactos eléctricos

Se observan picos angostos en torno a los 60 Hz y posiblemente en 120 Hz, los cuales corresponden a interferencia de la red eléctrica y sus armónicos. Esta presencia confirma que la señal aún no ha sido filtrada para suprimir el ruido ambiental.


## 5. Variabilidad entre canales

Cada línea del espectro corresponde a un canal EEG distinto. Se aprecia:

- mayor energía en canales frontales por parpadeos,
- ruido muscular de alta frecuencia en regiones temporales,
- canales más limpios en zonas centrales y parietales.

Esta variabilidad es típica en EEG sin preprocesar.


## 6. Relevancia para el procesamiento en MI

El análisis espectral permite justificar el empleo posterior de filtros IIR para separar las bandas de interés y el uso de un banco de filtros para aislar información relevante en el rango 8--30 Hz (alpha y beta), esencial en tareas de imaginación motora.

Asimismo, explica por qué el cálculo posterior de la FFT sobre los datos filtrados produce un arreglo de forma:

$$
(199,\;64,\;897,\;5),
$$

donde $897 = 1792/2 + 1$, consistente con la salida esperada de la transformada rFFT aplicada sobre el eje temporal.

In [ ]:
#espectro señales filtradas
Xwb = np.fft.rfft(Xrc,axis=2)

Xwb.shape

In [ ]:
#espectro señales filtradas por bandas - ritmos cerebrales

fig,ax = plt.subplots(5,1,figsize=(8,40))
# Graficar cada canal en un subplot banda respectiva
for b in range(f_bank.shape[0]): #bandas
    ax[b].plot(vfreq,20*np.log10(np.abs(Xwb[trial,:,:,b])).T)
    ax[b].set_xlabel('Frecuencia [Hz]')
    ax[b].set_ylabel('Magnitud [dB]')
    ax[b].set_title(f'Esepctro EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]}')
    
plt.show()

## Ejercicio 5

Discuta las gráficas

## Espectros filtrados por banda

1. Se remuestreó la señal a new_fs (256 Hz), obteniendo 1792 muestras por ensayo (7 s × 256 Hz).  
2. Se aplicó un banco de filtros definido por f_bank = np.array([[0.5,4.],[4., 8.],[8.,13.],[13.,32.],[32.,100.]])
4. A cada señal por trial y canal se le aplicó el filtro IIR correspondiente.  
5. Se calculó la transformada rápida de Fourier real (rFFT) sobre la dimensión temporal, obteniendo 1792/2+1 = 897 puntos frecuenciales por banda. Por esto Xwb.shape es (199,64,897,5).


## Delta (0.5-4 Hz)

- **Observación en la figura**: magnitudes muy altas en el extremo izquierdo del eje de frecuencia, con caída pronunciada hacia frecuencias mayores.  
- **Causa**: predominio de tendencias lentas, parpadeos (EOG) y artefactos de movimiento. Estas componentes concentran gran parte de la energía total en EEG crudo.  
- **Consecuencia práctica**: conviene atenuar delta (o quitarla) para análisis MI, ya que reduce SNR y puede enmascarar cambios en bandas motoras.


## Theta (4-8 Hz)

- **Observación**: pico concentrado en 4-8 Hz en varios canales; magnitud menor que en delta.  
- **Causa**: actividad relacionada con estados de somnolencia, memoria y ciertas dinámicas corticales.  
- **Consecuencia práctica**: theta aporta información cognitiva, pero no es la banda principal para MI; es importante controlarla para evitar sesgos en normalización.


## Alpha (8-13 Hz)

- **Observación**: pico claro en 8-13 Hz, más pronunciado en canales occipitales.  
- **Causa**: ritmo alfa típico (relajación, ojos cerrados). En MI suele observarse ERD/ERS en alfa sobre áreas motoras.  
- **Consecuencia práctica**: calcular potencia y cambios relativos en alfa por canal (p. ej. C3/C4) para detectar desincronización asociada a la imaginación motora.


## Beta (13-32 Hz)

- **Observación**: incremento de energía en 13-32 Hz con una forma más ancha y variable entre canales.  
- **Causa**: actividad sensoriomotora; también susceptible a contaminación por EMG.  
- **Consecuencia práctica**: beta es crítica para MI — usar bandpower y/o CSP en 13--30 Hz para clasificación.


## Gamma (32-100 Hz)

- **Observación**: energía distribuida en 32--100 Hz; pico estrecho frecuente en ~60 Hz (ruido de red).  
- **Causa**: gamma de superficie es débil y frecuentemente está contaminada por EMG y ruido eléctrico (60 Hz y armónicos).  
- **Consecuencia práctica**: evaluar si gamma aporta SNR útil; en muchos pipelines BCI se prioriza alpha/beta y se reduce gamma por su baja fiabilidad.


## Observaciones transversales y recomendaciones

1. La rFFT de 1792 muestras produce 897 bins frecuenciales, por eso la forma de la salida es (199,64,897,5)\).  
2. Aplicar **notch** en 50/60 Hz si aparecen picos estrechos de línea eléctrica; si se usa IIR, emplear \texttt{filtfilt} para evitar desfases de fase.  
3. Emplear **ICA** o regresión EOG para atenuar artefactos o rechazar trials con excesiva energía en delta/EMG.  
4. Normalizar potencias por trial y canal (p. ej. potencias logarítmicas o potencias relativas al baseline) para comparación entre sujetos.  
5. Para MI, extraer características en 8-30 Hz (alpha + beta): bandpower, envolventes Hilbert, CSP.  
6. Generar visualizaciones adicionales: mapas topográficos por banda, ERD/ERS en C3/C4, y comparación espectro crudo vs filtrado.

## Conclusión

Las figuras por banda confirman expectativas fisiológicas: delta domina las bajas frecuencias; alpha y beta muestran picos y energía relevantes para la imaginación motora; gamma es ruidosa y susceptible a artefactos; además se detecta contaminación de la red eléctrica (picos estrechos) y variabilidad intercanal. Estas observaciones definen las prioridades de preprocesamiento (notch, remoción EOG/EMG, filtrado cero-fase) y las bandas objetivo para extracción de características de MI.


## Visualización de espectrogramas

Consultar qué es la Short Time Fourier Transform



## Short-Time Fourier Transform (STFT)

La Transformada de Fourier de Tiempo Corto (STFT) es una herramienta que permite analizar como cambia el contenido frecuencial de una señal a lo largo del tiempo, a diferencia de la Transformada de Fourier tradicional, que proporciona información global en frecuencia, la STFT entrega información tiempo–frecuencia.

La idea fundamental consiste en dividir la señal en segmentos temporales cortos mediante una ventana w(t), y aplicar la Transformada de Fourier a cada segmento, asi se obtiene un espectro para cada instante de tiempo.

$$
\text{STFT}\{x(t)\}(t,\omega)
= \int_{-\infty}^{\infty} x(\tau)\, w(\tau - t)\, e^{-j\omega \tau}\, d\tau
$$

Donde:
- x(t) es la señal original.
- w(t) es una ventana localizada en el tiempo (por ejemplo Hamming, Hann, Gauss).
- t indica el desplazamiento temporal de la ventana.
- omega es la frecuencia angular.

Este proceso genera un mapa tiempo–frecuencia que usualmente se visualiza como un espectrograma, el cual se define como:

$$
\text{Espectrograma}(t,f) = \left| \text{STFT}(t,f) \right|^2
$$

**Caracteristicas de STFT**

La STFT esta enfocada a señales no estacionarias, es decir, señales cuyo contenido espectral cambia en el tiempo, por ejemplo:
- EEG y otras bioseñales.
- Señal de voz.
- Vibraciones.

La STFT está limitada por el principio de incertidumbre tiempo–frecuencia:
- Ventanas cortas proporcionan buena resolución temporal pero pobre resolución frecuencial.
- Ventanas largas ofrecen buena resolución frecuencial pero menor resolución temporal.

En resumen la STFT realiza la Transformada de Fourier sobre ventanas deslizantes de la señal, lo que permite conocer que frecuencias están presentes en cada instante del tiempo, esta es una herramienta fundamental para analizar señales no estacionarias.


In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
from scipy.signal import stft #
nperseg = 0.5*new_fs#longitud ventas en muestras
vfs,t,Xstft = stft(X,fs=new_fs,nperseg=nperseg,axis=2)
Xstft = 20*np.log10(abs(Xstft))

#graficar stft para un trial y un canal
trail = 0
chi = channels.index('C4')

fig, ax = plt.subplots(2, 1,figsize=(10,6))

ax[1].plot(tv,X[trail,chi,:])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstft[trail,chi])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Esepctrograma EEG Original -- Ch = {channels[chi]}')
print(Xstft.shape)

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 2
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Esepctrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')


# Ejercicio 6

Presente las gráficas de stft para distintos canales en los 5 ritmos cerebrales y discuta.

In [ ]:
# ----------------------------------------------
# EJERCICIO 6: STFT para los ritmos cerebrales
# ----------------------------------------------

from scipy.signal import stft
import matplotlib.pyplot as plt
import numpy as np

# parámetros usados en el notebook
nperseg = 256        # ventana STFT
trail = 10           # trial de ejemplo
channels_to_plot = [0, 10, 20]   # puedes cambiar los canales
ritmo = ["delta","theta","alpha","beta","gamma"]

print("Canales a graficar:", [channels[ch] for ch in channels_to_plot])
print("Ritmos:", ritmo)

for b in range(5):  # 5 bandas filtradas
    for chi in channels_to_plot:

        # aplicar STFT sobre el eje temporal de Xrc
        vfs, t, Xstft_b = stft(Xrc, fs=new_fs, nperseg=nperseg, axis=2)
        Xstft_b = 20*np.log10(abs(Xstft_b))   # magnitud en dB

        fig, ax = plt.subplots(2, 1, figsize=(12,7))

        # señal temporal filtrada
        ax[1].plot(tv, Xrc[trail, chi, :, b])
        ax[1].set_ylabel("Amp. [$\mu$V]")
        ax[1].set_title(f"Señal temporal filtrada – Canal {channels[chi]} – Ritmo {ritmo[b]}")

        # espectrograma
        im = ax[0].pcolormesh(t, vfs, Xstft_b[trail, chi, :, b, :], shading='gouraud')
        fig.colorbar(im, ax=ax[0], orientation="horizontal", pad=0.25)

        ax[0].set_ylabel("Frecuencia [Hz]")
        ax[0].set_xlabel("Tiempo [s]")
        ax[0].set_title(
            f"Espectrograma EEG Filtrado {f_bank[b,0]}–{f_bank[b,1]} Hz – Ritmo {ritmo[b]} – Canal {channels[chi]}"
        )

        plt.tight_layout()
        plt.show()


En este ejercicio analizamos varios canales del EEG aplicando la STFT sobre las señales ya filtradas mediante el banco de filtros IIR. A partir de los espectrogramas obtenidos podemos destacar los siguientes puntos:

# 1. Ritmo delta (0.5–4 Hz)
Este ritmo muestra concentraciones de energía en frecuencias muy bajas, lo cual se observa como zonas de alta intensidad en la parte inferior del espectrograma. La energía se mantiene estable en el tiempo, característica típica de actividad de sueño profundo y procesos de sincronización cortical lenta.

# 2. Ritmo theta (4–8 Hz)
Los espectrogramas exhiben mayor actividad en la banda theta, apreciándose picos de energía intermitentes. Este comportamiento suele relacionarse con estados de relajación, somnolencia o procesamiento de memoria.

# 3. Ritmo alpha (8–13 Hz)
El ritmo alpha muestra una concentración clara de energía alrededor de 10 Hz, particularmente marcada en regiones occipitales. Esta banda se asocia a estados de relajación con los ojos cerrados y disminuye cuando el sujeto está realizando una tarea cognitiva activa.

# 4. Ritmo beta (13–32 Hz)
Los espectrogramas en esta banda presentan energía distribuida entre 15–25 Hz, con variaciones más rápidas en el tiempo. El ritmo beta está relacionado con actividad motora, atención y procesos cognitivos más exigentes. La mayor variabilidad temporal concuerda con la naturaleza más activa de este ritmo.

# 5. Ritmo gamma (32–100 Hz)
El espectrograma gamma exhibe contenido energético en frecuencias altas con gran variabilidad temporal. Esta banda se vincula con procesos de integración sensorial, atención sostenida y actividad cognitiva compleja. Las variaciones rápidas observadas son típicas de este ritmo.

En conclusión los espectrogramas confirman que los filtros IIR aplicados al EEG separan adecuadamente las bandas de interés, y la STFT permite evidenciar cómo varía la energía en cada frecuencia a lo largo del tiempo. Cada ritmo cerebral presenta una firma tiempo frecuencia característica que coincide con la fisiología conocida del EEG:

- ritmos lentos (delta, theta) → energía estable y concentrada en bajas frecuencias,  
- ritmos intermedios (alpha, beta) → energía focalizada pero con mayor dinámica,  
- ritmos rápidos (gamma) → actividad altamente variable y extendida en altas frecuencias.

En conjunto, las gráficas de STFT permiten observar de forma clara la evolución temporal de cada ritmo y corroborar la correcta segmentación frecuencial realizada por el banco de filtros.


## Visualización de señales EEG sobre montaje 10-20

In [ ]:
import mne

# Cargar el montaje estándar
easycap_montage = mne.channels.make_standard_montage("standard_1020")


# Crear un montaje personalizado con los electrodos seleccionados
custom_pos = {ch: easycap_montage.get_positions()["ch_pos"][ch] for ch in channels}
custom_montage = mne.channels.make_dig_montage(ch_pos=custom_pos, coord_frame="head")

# Mostrar el montaje personalizado
custom_montage.plot(show_names=True)
fig = custom_montage.plot(kind="3d", show_names=True, show=False)
fig.gca().view_init(azim=70, elev=15)  # Ajustar la vista 3D

In [ ]:
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.visualizations.git

# Topomaps

In [ ]:
from gcpds.visualizations.topoplots import topoplot


trial = 150
vec_topo_o = abs(X[trial,:]).mean(axis=-1)
vec_topo_b = abs(Xrc[trial,:,:,:]).mean(axis=1)


fig,ax = plt.subplots(1,6,figsize=(20,10))
topoplot(vec_topo_o, channels, contours=3, cmap='Reds', names=channels, sensors=False,ax=ax[0],show=False,vlim=(min(vec_topo_o), max(vec_topo_o)))

for b in range(f_bank.shape[0]):
    vec_ = vec_topo_b[:,b]
    topoplot(vec_, channels, contours=3, cmap='Reds', names=channels, sensors=False,ax=ax[b+1],show=False,vlim=(min(vec_), max(vec_)))
    ax[b+1].set_title(ritmo[b])    

ax[0].set_title(f'EEG-suj={sbj}-trial={trial}')    

plt.show()

## Ejercicio 7

Discuta

Tenemos seis mapas topográficos el primero correspondiente al EEG original y los siguientes cinco correspondientes a las señales filtradas en las bandas del banco f_bank, estos mapas representan la distribución espacial de la amplitud promedio en un trial específico, permitiendo identificar qué regiones corticales presentan mayor actividad en cada ritmo cerebral.


## 1. EEG original

El mapa del EEG crudo muestra una mezcla simultánea de todas las bandas de frecuencia.
Se observa potencia elevada en regiones frontales (Fp1–Fp2), característica de artefactos
oculares y actividad lenta. La distribución espacial no permite identificar claramente
ritmos neurales específicos debido a la superposición de componentes espectrales y ruido.

## 2. Ritmo Delta (0.5-4 Hz)

El topoplot delta presenta actividad predominante en regiones frontales y prefrontales.
El ritmo delta está fuertemente influenciado por artefactos fisiológicos de baja frecuencia
(parpadeos, movimientos, deriva de línea base). Su distribución carece de focalización
cortical clara, siendo un ritmo poco informativo para tareas de imaginación motora.

## 3. Ritmo Theta (4-8 Hz)

En theta se observa actividad moderada en zonas frontocentrales. Este ritmo se asocia a
procesos de atención, memoria y estados de somnolencia. Su expresión topográfica depende
del estado cognitivo del sujeto, pero no presenta un patrón motor marcado.

## 4. Ritmo Alpha (8-13 Hz)

El ritmo alpha muestra una concentración marcada en regiones occipitales, consistente con
la literatura neurofisiológica. Este patrón aumenta en estados de relajación con ojos 
cerrados y tiende a disminuir durante la ejecución o imaginación de tareas motoras. El 
mapa alpha permite identificar modulaciones del ritmo occipital y posibles efectos ERD/ERS.

## 5. Ritmo Beta (13-32 Hz)

El topoplot beta es el más relevante para la imaginación motora. Se observa actividad 
focalizada en las áreas sensoriomotoras (regiones C3 y C4). La topografía beta puede 
presentar asimetría según la mano imaginada, reflejando desincronización (ERD) en la 
corteza motora contralateral. Esto lo convierte en un ritmo fundamental para sistemas BCI.

## 6. Ritmo Gamma (32-100 Hz)

El mapa gamma presenta actividad distribuida en regiones temporales y zonas 
susceptibles a contaminación muscular (EMG). El ritmo gamma en EEG de superficie tiene 
relativamente bajo cociente señal–ruido y es altamente sensible a artefactos y ruido 
eléctrico. Su patrón espacial suele ser menos estable que el de bandas más bajas.

## Conclusiones

Los mapas topográficos muestran que cada ritmo cerebral posee una distribución espacial 
característica:

- Delta y theta: actividad lenta de origen no específico, altamente influenciada por artefactos.  
- Alpha: actividad occipital típica.  
- Beta: activación sensoriomotora crucial para imaginación motora.  
- Gamma: actividad rápida con alta susceptibilidad al ruido.

La comparación entre el EEG original y los mapas filtrados confirma que el banco de 
filtros permite aislar correctamente los ritmos corticales y facilita el análisis espacial 
en tareas de BCI basadas en imaginación motora.


## Common Spatial Patterns

Consulté qué son los Common Spatial Patterns (CSP) y su aplicación al procesado de señales EEG

# Common Spatial Patterns (CSP) y su aplicación al procesado de señales EEG

Los Common Spatial Patterns (CSP) son un método de filtrado espacial 
supervisado ampliamente utilizado en el procesado de señales EEG, especialmente en sistemas de Interfaces Cerebro-Computador (BCI) basados
en tareas de imaginación motora. El objetivo de CSP es encontrar combinaciones lineales de los canales EEG que maximicen la separabilidad entre dos clases.

**¿Qué hace CSP?**

CSP busca filtros espaciales que generen nuevas proyecciones de la señal 
donde:
- una clase presenta \textbf{máxima varianza}, y  
- la otra clase presenta \textbf{mínima varianza}.  
La varianza de una señal EEG es proporcional a su \textbf{potencia}, por lo que 
CSP explota diferencias en la actividad cortical entre clases. En tareas de 
imaginación motora, por ejemplo, la potencia en las bandas mu y beta disminuye 
de manera diferencial entre regiones C3 y C4 según la mano imaginada. CSP 
capta exactamente estas diferencias.

**Interpretación neurofisiológica**

Cada filtro CSP produce un mapa espacial que representa pesos positivos y 
negativos asociados a cada electrodo:
- Pesos positivos: aumento relativo de actividad.  
- Pesos negativos: disminución relativa.  

Los mapas de CSP suelen resaltar:
- la \textbf{corteza sensoriomotora},  
- patrones de lateralización (C3 vs. C4),  
- zonas relevantes para eventos ERD/ERS.  

Esto permite interpretar de manera fisiológica qué áreas discriminan mejor 
entre dos clases de movimiento imaginado.

**Formulación matemática**

Sea una matriz de datos de EEG con C canales y T muestras:
$$
X_1 \in \mathbb{R}^{C \times T}, \qquad 
X_2 \in \mathbb{R}^{C \times T}
$$
correspondientes a dos clases:

1. Se calculan las matrices de covarianza normalizadas:
$$
R_1 = \frac{X_1 X_1^\top}{\operatorname{trace}(X_1 X_1^\top)}, 
\qquad
R_2 = \frac{X_2 X_2^\top}{\operatorname{trace}(X_2 X_2^\top)}.
$$

2. Se forma la matriz de covarianza compuesta:
$$
R = R_1 + R_2.
$$

3. Se realiza la descomposición espectral:
$$
R = U \Lambda U^\top.
$$

4. Se construye la matriz de blanqueamiento:
$$
P = \Lambda^{-1/2} U^\top.
$$

5. Se diagonaliza simultáneamente:
$$
S_1 = P R_1 P^\top = B \Lambda_1 B^\top.
$$
Las columnas de B forman los filtros espaciales CSP.
Los primeros filtros maximizan la varianza de la clase 1 y minimizan la de la clase 2. 
Los últimos filtros hacen lo contrario.

**Extracción de características**

Tras aplicar los filtros:
$$
Z = W X,
$$

las características se obtienen mediante la varianza logarítmica:
$$
f_i = 
\log 
\left( 
\frac{\operatorname{var}(Z_i)}
{\sum_j \operatorname{var}(Z_j)} 
\right).
$$

Estos vectores de características suelen emplearse junto a clasificadores como:
- LDA,
- SVM,
- Random Forest.

**Aplicación de CSP en BCI**

CSP es uno de los métodos más utilizados en:
- competiciones BCI,
- análisis de imaginación motora (MI),
- bases de datos como BCI Competition II/III/IV,
- software especializado como MNE, BCI2000, OpenViBE.

Su eficacia deriva de que los ritmos mu (8--13 Hz) y beta (13--32 Hz) presentan 
modulaciones espaciales claras (ERD/ERS) durante MI, especialmente en las áreas 
motoras contralaterales.

**Ventajas**

- Alta capacidad discriminativa para dos clases.  
- Fácil de interpretar mediante topoplots.  
- Computacionalmente eficiente.  
- Excelente desempeño en MI.

**Desventajas**

- Sensible a ruido y artefactos (EOG, EMG).  
- Requiere estabilidad en los electrodos.  
- La versión estándar sólo maneja dos clases 
  (extensiones: One-vs-Rest, Multi-CSP, FBCSP).

**Resumen**
$$
\textbf{CSP} \Rightarrow
\text{ filtros espaciales que maximizan diferencias de potencia entre clases}.
$$

Es un método fundamental para separar patrones de actividad cortical en 
imaginación motora y constituye una de las técnicas más importantes dentro 
del preprocesamiento y extracción de características en BCI.


In [ ]:
import mne
from mne.decoding import CSP

# Instancia del objeto CSP
n_components = 2
csp = CSP(n_components=n_components, log= True, transform_into='average_power')
# Ajuste y transformación de los datos
csp_data = csp.fit_transform(X.astype(np.float64), y)

In [ ]:
print("CSP Transformado Shape:", csp_data.shape)
plt.scatter(csp_data[:,0],csp_data[:,1],c=y)
plt.show()

In [ ]:
#EEG original
fig,ax = plt.subplots(1,n_components,figsize=(5,5))
for cc in range(n_components):
    vec_ = np.abs(csp.filters_[cc])
    topoplot(vec_, channels, contours=3, cmap='Reds', names=channels, sensors=False,ax=ax[cc],show=False,vlim=(min(vec_), max(vec_)))
    ax[cc].set_title(f'CSP {cc+1}') 


In [ ]:
#lectura de datos
sbj = 14
X, y = load_GIGA(sbj=sbj, **load_args)

f_bank = np.array([[0.5,4.],[4., 8.],[8.,13.],[13.,32.],[32.,100.]])
vwt = np.array([[0.25, 1.75],[1.5,3],[2.75,4.25],[4,5.5],[5.25,6.75]]) #2.5 - 5 MI 0 - 7 trial completo
tf_repr = TimeFrequencyRpr(sfreq = new_fs, f_bank = f_bank,vwt=vwt)
X_ = np.squeeze(tf_repr.transform(X))
X_.shape

In [ ]:
# csp por ventanas y ritmos
# Definir las dimensiones del arreglo
ritmos_ = f_bank.shape[0] 
ventanas_ = vwt.shape[0]
n_comp = 2
# Inicializar el arreglo vacío con listas anidadas
csp_M = [[None for _ in range(ventanas_)] for _ in range(ritmos_)]
csp_filters_ = np.zeros((ritmos_,ventanas_,X_.shape[1],X_.shape[1])) #ritmos ventanas Ch
Xcsp_ = np.zeros((X_.shape[0],n_comp,ritmos_,ventanas_))

for i in range(ritmos_):
    for j in range(ventanas_):
        print(f'CSP ritmo {f_bank[i]} -- ventana {vwt[j]}...')
        csp_M[i][j] =  CSP(n_components=n_comp, log= True, transform_into='average_power')
        Xcsp_[:,:,i,j] = csp.fit_transform(X_[:,:,:,j,i].astype(np.float64), y)
        csp_filters_[i,j,:] = np.abs(csp.filters_) 

In [ ]:
# graficar topomaps
fig, ax = plt.subplots(ritmos_,ventanas_,figsize=(12,12))

for i in range(ritmos_):
    for j in range(ventanas_):
        vec_ = csp_filters_[i,j,0]
        vec_ = vec_/max(vec_)
        topoplot(vec_, channels, contours=3, cmap='Reds', names=None, sensors=False,ax=ax[i,j],show=False,vlim=(min(vec_), max(vec_)))
    ax[i,0].set_ylabel(ritmo[i],fontsize=20)   
for j in range(ventanas_):
     ax[0,j].set_title(f'{vwt[j,0]}--{vwt[j,1]} [s]',fontsize=15)
    
plt.subplots_adjust(hspace=-0.025,wspace=-0.025)    
plt.show()      

In [ ]:
#scatters
fig, ax = plt.subplots(ritmos_,ventanas_,figsize=(12,12))

for i in range(ritmos_):
    for j in range(ventanas_):
        ax[i,j].scatter(Xcsp_[:,0,i,j],Xcsp_[:,1,i,j],c=y)
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])
    ax[i,0].set_ylabel(ritmo[i],fontsize=20)   
for j in range(ventanas_):
     ax[0,j].set_title(f'{vwt[j,0]}--{vwt[j,1]} [s]',fontsize=15)
    
plt.subplots_adjust(hspace=0.1,wspace=0.1)    
plt.show()  

## Nueva implementacion punto 2.3

Adaptar y modificar el codigo base del cuaderno del punto 2.1 para mejorar la deteccion de patrones de imaginacion motora en distintos sujetos y condiciones experimentales. Discuta las dificultades encontradas respecto a lo estudiado en el estado del arte,especialmente, respecto a la inconsistencia en la identificacion de patrones espaciales, temporales y frecuenciales en estas señales entre sujetos y estımulos.

In [ ]:
import numpy as np
from scipy.signal import butter, sosfiltfilt

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Métodos riemannianos (mucho más robustos que CSP)
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

# Filtro notch para eliminar ruido de red 50 Hz (artefacto experimental)
from mne.filter import notch_filter


# SAFE WINDOW EXTRACTION (VERSIÓN ROBUSTA)
def safe_extract_window(xf, sfreq, t0, t1):
    """
    EXTRACCIÓN SEGURA DE VENTANAS TEMPORALES.
    - El cuaderno original usaba ventanas relativas al inicio del ensayo,
      pero estas ventanas a veces quedaban fuera del rango de la señal,
      generando segmentos vacíos o matrices de covarianza mal definidas.
    - Este método evita ventanas vacías alineando la ventana pedida a los
      límites reales de la señal.
    - Siempre garantiza que la ventana tiene el mismo tamaño.
    """

    n_ch, n_samples = xf.shape
    win_len = int((t1 - t0) * sfreq)   # longitud fija de la ventana

    # Creamos ventana destino llena de ceros
    seg = np.zeros((n_ch, win_len), dtype=np.float32)

    # Índices reales en muestras
    start = int(t0 * sfreq)
    end = start + win_len

    # Clipping para no salirnos de la señal
    s_in = max(start, 0)
    e_in = min(end, n_samples)

    # Alineación dentro de la ventana generada
    s_out = s_in - start
    e_out = s_out + (e_in - s_in)

    # Copiamos el segmento disponible
    if e_in > s_in:
        seg[:, s_out:e_out] = xf[:, s_in:e_in]

    return seg


# REPRESENTACIÓN TIEMPO–FRECUENCIA
class TimeFrequencyRpr:
    """
    Esta clase reemplaza la extracción temporal simple del cuaderno original
    por una representación tiempo–frecuencia robusta y multiparámetro.

    Mejoras:
    - Se usa un banco de filtros multibanda (θ, μ, β baja, β alta).
    - Se aplican varias ventanas temporales desfasadas.
    - Se incluye un notch filter, que combate ruido experimental
      (parpadeo de luz, interferencia eléctrica).
    - Entrega un tensor 5D (trial × canal × tiempo × ventana × banda).
    """

    def __init__(self, sfreq, f_bank, vwt, filt_order=4):
        self.sfreq = float(sfreq)
        self.f_bank = np.array(f_bank) # banco de filtros
        self.vwt = np.array(vwt)       # ventanas temporales
        self.filt_order = filt_order

    def _bandpass(self, x, low, high):
        """Bandpass estable con filtro Butterworth."""
        nyq = 0.5 * self.sfreq
        sos = butter(
            self.filt_order,
            [low / nyq, high / nyq],
            btype="bandpass",
            output="sos",
        )
        return sosfiltfilt(sos, x, axis=-1)

    def transform(self, X):
        X = np.asarray(X)
        n_trials, n_ch, n_samples = X.shape
        n_bands = len(self.f_bank)
        n_windows = len(self.vwt)

        # Longitud máxima entre todas las ventanas
        win_len = max(int((t1 - t0) * self.sfreq) for t0, t1 in self.vwt)

        # Tensor 5D de salida
        out = np.zeros((n_trials, n_ch, win_len, n_windows, n_bands),
                       dtype=np.float32)

        for b_idx, (f0, f1) in enumerate(self.f_bank):
            for t in range(n_trials):

                # --- Eliminamos ruido de la red eléctrica ---
                xf = notch_filter(X[t], self.sfreq, freqs=50)

                # --- Aplicamos filtro bandpass ---
                xf = self._bandpass(xf, f0, f1)

                # --- Extraemos cada ventana de tiempo ---
                for w_idx, (t0, t1) in enumerate(self.vwt):
                    seg = safe_extract_window(xf, self.sfreq, t0, t1)
                    out[t, :, :, w_idx, b_idx] = seg

        return out


# NORMALIZACIÓN POR ENSAYO
def center_scale_trials(X):
    """
    Normalización por ensayo:
    - Quita el promedio por canal.
    - Escala por desviación estándar.
    Justificación:
    - Reduce variabilidad inter-sujeto e inter-sesión.
    - Evita que diferencias de amplitud dominen las covarianzas.
    """
    X = X - X.mean(axis=2, keepdims=True)
    X = X / (X.std(axis=2, keepdims=True) + 1e-6)
    return X


# CARGA DE UN SUJETO Y UNA SESIÓN
def load_subject_tf(sbj, tf_repr, load_args=None):
    """
    Mejoras:
    - Se incorpora el concepto de sesión para capturar variabilidad
      experimental (fatiga, ruido, impedancia, ambiente, etc.).
    - Cada sujeto puede tener varias sesiones → mayor robustez
      cross-subject + cross-session.
    """
    if load_args is None:
        load_args = {}

    # Carga real del dataset
    X, y = load_GIGA(sbj=sbj, session=session, **load_args)

    X = center_scale_trials(X)
    X_tf = tf_repr.transform(X)
    return X_tf, np.asarray(y)


# FILTERBANK RIEMANNIANO
class FilterBankRiemann(BaseEstimator, TransformerMixin):
    """
    Este módulo reemplaza al CSP clásico del cuaderno original.
    - CSP falla con matrices mal condicionadas.
    - Los métodos riemannianos son estado del arte para cross-subject.
    - Tangent Space transforma covarianzas en features lineales estables.
    - Se usa Regularización OAS para evitar singularidades.
    """

    def __init__(self):
        pass

    def fit(self, X, y):
        X = np.asarray(X)
        n_trials, n_ch, n_time, n_windows, n_bands = X.shape

        self.models_ = []

        for b in range(n_bands):
            for w in range(n_windows):

                # Datos de una banda y una ventana
                data = X[:, :, :, w, b].astype(np.float64)

                # Regularización OAS = más estable que Ledoit-Wolf y sin fallos
                cov = Covariances(estimator='oas').fit_transform(data)

                # Ajuste del espacio tangente
                ts = TangentSpace().fit(cov, y)

                self.models_.append((b, w, ts))

        return self

    def transform(self, X):
        feats = []

        for b, w, ts in self.models_:
            data = X[:, :, :, w, b].astype(np.float64)

            # Covarianza regularizada
            cov = Covariances(estimator='oas').transform(data)

            # Transformación al espacio tangente
            feats.append(ts.transform(cov))

        # Concatenación final de features riemannianos
        return np.concatenate(feats, axis=1)


# CONFIGURACIÓN DE SUJETOS Y SESIONES
sbj_list = list(range(1, 6))  # sujetos
session_list = [1, 2]         # múltiples sesiones → variabilidad experimental

# Banco de filtros extendido → variabilidad frecuencial inter-sujeto
f_bank = np.array([
    [4, 8],    # theta
    [8, 13],   # mu
    [13, 26],  # beta baja
    [26, 35],  # beta alta
])

# Ventanas temporales desplazadas → variabilidad temporal
vwt = np.array([
    [0.0, 2.0],
    [1.0, 3.0],
    [2.0, 4.0],
])

# Transformación tiempo–frecuencia robusta
tf_repr = TimeFrequencyRpr(
    sfreq=new_fs,
    f_bank=f_bank,
    vwt=vwt
)

# CARGA COMPLETA DEL DATASET (SUJETOS × SESIONES)
X_all, y_all, groups = [], [], []

"""
Se crea un vector 'groups' que identifica cada combinación sujeto-sesión.
Esto permite una validación realista cross-subject + cross-session,
al contrario del cuaderno original que solo aislaba sujetos.
"""

for sbj in sbj_list:
    for sess in session_list:
        Xi, yi = load_subject_tf(sbj, sess, tf_repr, load_args)
        X_all.append(Xi)
        y_all.append(yi)
        groups.extend([f"{sbj}_sess{sess}"] * len(yi))

X_all = np.concatenate(X_all)
y_all = np.concatenate(y_all)
groups = np.asarray(groups)

print("X_all:", X_all.shape)

# PIPELINE FINAL
clf = Pipeline(steps=[
    ("fb", FilterBankRiemann()),   # extracción de features riemannianos
    ("scaler", StandardScaler()),  # normalización a nivel de feature
    ("logreg", LogisticRegression(
        max_iter=1000,
        class_weight="balanced"   # corrige desbalance entre clases
    )),
])
# VALIDACIÓN CROSS-SUBJECT + CROSS-SESSION
"""
GroupKFold garantiza que todo un SUJETO-SESIÓN esté completamente
fuera del entrenamiento durante cada fold, evaluando generalización real.

Esto es un requerimiento del proyecto: evaluar variabilidad inter-sujeto
e inter-condición experimental.
"""

n_splits = max(2, min(5, len(np.unique(groups))))
gkf = GroupKFold(n_splits=n_splits)

scores = cross_val_score(
    clf,
    X_all,
    y_all,
    groups=groups,
    cv=gkf,
    scoring="roc_auc"
)

print("AUC por fold:", np.round(scores, 3))
print("AUC promedio:", np.round(scores.mean(), 3))


El desarrollo del ejercicio 2.3 nos permite profundizar en los principales 
desafíos asociados al reconocimiento de patrones de imaginación motora (MI)  en señales EEG multisucesor y bajo diferentes condiciones experimentales. Los resultados obtenidos tras adaptar el cuaderno base del punto 2.1 coinciden ampliamente con lo reportado en el estado del arte sobre sistemas BCI basados en MI, especialmente en configuraciones de tipo subject–independent y cross-session, donde la variabilidad fisiológica y experimental se manifiesta con mayor intensidad. A continuación se discuten los hallazgos más relevantes y las modificaciones implementadas al código original para mitigar dichas dificultades.

**Variabilidad espacial entre sujetos**

La literatura señala que los patrones ERD/ERS asociados a MI se concentran principalmente en las regiones motoras y sensorimotores (e.g., C3, C4 y Cz). Sin embargo, la distribución espacial exacta presenta variabilidad inter-sujeto debido a factores como diferencias anatómicas, variabilidad en la posición de electrodos, diferencias en la conductividad del cráneo y diversidad en la ejecución mental de la tarea. 

En el cuaderno original, esta variabilidad generó matrices de covarianza mal condicionadas para ciertos sujetos, lo que ocasionó fallos numéricos en métodos como CSP, que dependen de la resolución estable de problemas de autovalores generalizados. Algunas matrices incluso dejaron de ser definidas positivas, haciendo imposible la descomposición requerida por el algoritmo.

Para abordar este problema, se reemplazó CSP por un enfoque riemanniano basado en Covariances y proyección al Tangent Space.Este método es más robusto ante variaciones en la topografía cortical, tolera mejor el ruido espacial y evita la necesidad de invertir matrices mal condicionadas. Esta elección está en línea con lo reportado en competiciones como BCI-IV y BNCI Horizon, donde los métodos riemannianos han demostrado ser superiores en escenarios con alta variabilidad espacial.

**Variabilidad temporal y corrección del alineamiento**

El cuaderno base empleaba ventanas temporales fijas y centradas en el inicio del ensayo, lo que provocó que muchas ventanas quedaran parcial o totalmente fuera del rango válido de señal para varios sujetos. Esta incongruencia se origina en la variabilidad temporal inherente de la respuesta MI: el inicio del fenómeno ERD/ERS no es constante entre sujetos ni entre ensayos, y puede desplazarse varios cientos de milisegundos hacia adelante o atrás dependiendo del nivel de atención, fatiga o experiencia del participante.

Para corregir esta limitación, se implementó un método robusto de extracción temporal denominado \textit{safe clipping}, el cual garantiza que cada ventana tenga una longitud fija independientemente de su posición relativa.Esto evita la generación de segmentos vacíos o de longitud cero, corrige uno de los problemas más críticos del cuaderno inicial y permite construir matrices de covarianza estables en todos los escenarios evaluados.

**Variabilidad frecuencial y debilidad del ERD/ERS**

Aunque la MI está asociada típicamente a desincronización en las bandas (8-13 Hz) y (13-30 Hz), numerosos estudios reportan que la distribución espectral exacta del ERD/ERS varía significativamente entre sujetos. Esta variabilidad frecuencial se intensifica en sujetos poco entrenados o en condiciones experimentales ruidosas (p. ej., tensión muscular,parpadeo o fluctuaciones en la concentración).

El cuaderno base utilizaba pocas bandas, lo que reducía la capacidad de capturar patrones alternos. Para mitigar este límite, se amplió el banco de filtros incluyendo bandas $\theta$, $\mu$ y dos subbandas $\beta$. Esto aumenta la sensibilidad a la variabilidad inter-sujeto e inter-estímulo, permitiendo representar la actividad motora imaginada con mayor flexibilidad.

**Influencia de las condiciones experimentales**

Además de la variabilidad fisiológica entre sujetos, las condiciones experimentales introducen cambios significativos en la señal: diferencias en sesiones, en la impedancia de electrodos, ruido ambiental, fatiga del sujeto,micro-movimientos musculares y fluctuaciones de atención. Estos factores alteran la dinámica temporal y espectral del EEG, generando cambios en la amplitud del ERD/ERS entre sesiones y provocando que un modelo entrenado en una sesión falle en generalizar a otra.

Para capturar esta variabilidad, se reorganizaron los grupos de validación incluyendo no sólo la identidad del sujeto, sino también su sesión o condición experimental. Esta estrategia permitió evaluar la capacidad real del modelo para generalizar más allá de diferencias fisiológicas entre sujetos, incorporando las variaciones adicionales introducidas por las condiciones de registro.

**Justificación del enfoque riemanniano frente a CSP***

Las dificultades observadas en las etapas espacial, temporal y frecuencial confirman que métodos clásicos como CSP presentan limitaciones fuertes en contextos de alta variabilidad inter-sujeto y cross-session. Por el contrario, los enfoques riemannianos operan directamente sobre la geometría del espacio de matrices de covarianza, evitando la inversión explícita de matrices inestables y proporcionando representaciones robustas incluso ante diferencias significativas en energía, ruido o dispersión espacial de los patrones.

El uso de bank filtering junto con el mapeo al espacio tangente constituye un pipeline más estable, consistente y alineado con el estado del arte, permitiendo una detección más fiable de patrones MI en escenarios complejos.


En conclusiòn la adaptación del código base del cuaderno nos permitió identificar y corregir limitaciones importantes relacionadas con la variabilidad espacial, temporal y frecuencial propia del EEG, así como con las condiciones experimentales del registro. Las modificaciones implementadas—ventanas temporales robustas,representación tiempo–frecuencia ampliada y el reemplazo de CSP por métodos riemannianos—proporcionan una solución más estable para la detección de MI en configuraciones cross-subject y cross-session. Este pipeline constituye una alternativa más adecuada y cercana al estado del arte para aplicaciones reales de interacción cerebro–computador.
